In [266]:
# pip install openpyxl python-pptx

In [267]:
import openpyxl
from pptx import Presentation

def find_table(slide):
    for shape in slide.shapes:
        if shape.has_table:
            return shape.table
    return None


In [268]:

# Map Excel headers to (row, col) in the PowerPoint table
excel_to_table_map = {
    "PS No": (0, 1),
    "Name": (0, 4),
    "DOB | Age": (1, 1),
    "Designation": (1, 4),
    "Site | Office": (2, 1),
    "IC | Location": (2, 4),
    "DOJ": (3, 1),
    "IS Name & Grade": (3, 4),
    "Grade": (4, 1),
    "Last Promotion Date": (4, 4),
    "DOR": (5, 1),
    "FAIR 23|24|25": (5, 3), # This is not in the excel file, but in the PPT
    "CP Rating 2025": (5, 6),
    "Total Yrs. of Exp| L&T Exp": (6, 1),
    "DC Score| Year": (6, 4),
    "Qualification | Year": (7, 1),
    "Current Role": (9, 0),  # This is not in the excel file, but in the PPT
    "Role Envisaged in 2025-26 (in Terms of Job Rotation/Job Enlargement)": (9, 3), # This is not in the excel file, but in the PPT
}

In [269]:
# Load Excel with data_only=True to get calculated values
wb = openpyxl.load_workbook('Excel-Test.xlsx', data_only=True)
ws = wb.active
excel_headers = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
print("Excel Headers:", excel_headers)

Excel Headers: ['#', 'PS No', 'Name', 'DOB', 'Age', 'DOB | Age', 'Designation', 'Site | Office', 'IC', 'Location', 'IC | Location', 'DOJ', 'DOH', 'GET-DET', 'IS Name', 'IS Grade', 'IS Name & Grade', 'Grade', 'Last Promotion Date', 'DOR', 'Prev. Rating\n(22/23/24)', 'Rating 2025', 'CP Rating 2025', 'Total Years of Experience', 'L&T Experience', 'Total Yrs. of Exp| L&T Exp', 'DC Score| Year', 'Qualification | Year', 'Role_Inputs']


In [270]:
# Function to print table content
def print_table_content(table):
    for row_idx, row in enumerate(table.rows):
        for col_idx, cell in enumerate(row.cells):
            print(f"Row {row_idx}, Col {col_idx}: {cell.text}")

# Usage


In [271]:
# Load PowerPoint template
prs = Presentation('Role.pptx')
template_slide = prs.slides[0]
table = find_table(template_slide)
print_table_content(table)

Row 0, Col 0: PS No
Row 0, Col 1: 
Row 0, Col 2: Name
Row 0, Col 3: 
Row 0, Col 4: 
Row 0, Col 5: PHOTO

Row 0, Col 6: 
Row 1, Col 0: DOB | Age
Row 1, Col 1: 
Row 1, Col 2: Designation
Row 1, Col 3: 
Row 1, Col 4: 
Row 1, Col 5: 
Row 1, Col 6: 
Row 2, Col 0: Site | Office 
Row 2, Col 1: 
Row 2, Col 2: IC | Location
Row 2, Col 3: 
Row 2, Col 4: 
Row 2, Col 5: 
Row 2, Col 6: 
Row 3, Col 0: DOJ
Row 3, Col 1: 
Row 3, Col 2: IS Name & Grade
Row 3, Col 3: 
Row 3, Col 4: 
Row 3, Col 5: 
Row 3, Col 6: 
Row 4, Col 0: Grade
Row 4, Col 1: 
Row 4, Col 2: Last Promotion Date
Row 4, Col 3: 
Row 4, Col 4: 
Row 4, Col 5: 
Row 4, Col 6: 
Row 5, Col 0: DOR
Row 5, Col 1: 
Row 5, Col 2: FAIR 23|24|25
Row 5, Col 3: 
Row 5, Col 4: 
Row 5, Col 5: CP Rating 2025
Row 5, Col 6: 
Row 6, Col 0: Total Yrs. of Exp| L&T Exp
Row 6, Col 1: 
Row 6, Col 2: DC Score| Year
Row 6, Col 3: 
Row 6, Col 4:  
Row 6, Col 5: 
Row 6, Col 6: 
Row 7, Col 0: Qualification | Year
Row 7, Col 1:  
Row 7, Col 2: 
Row 7, Col 3: 
Row 7, Co

In [272]:
def print_slide_layout_content(slide_layout):
    print(f"Slide Layout Name: {slide_layout.name}")
    print("Placeholders:")
    for placeholder in slide_layout.placeholders:
        print(f"  Placeholder idx: {placeholder.placeholder_format.idx}, type: {placeholder.placeholder_format.type}")
    print("Shapes:")
    for shape in slide_layout.shapes:
        print(f"  Shape name: {shape.name}, type: {shape.shape_type}")

In [273]:
import copy
from copy import deepcopy
def copy_slide(prs, slide):
    # Use the same layout as the source slide
    slide_layout = slide.slide_layout
    new_slide = prs.slides.add_slide(slide_layout)
    for shape in slide.shapes:
        el = copy.deepcopy(shape.element)
        new_slide.shapes._spTree.insert_element_before(el, 'p:extLst')
    return new_slide

In [274]:
from datetime import datetime

def format_date(date_str):
    date_str = str(date_str).strip()
    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%d"):
        try:
            dt = datetime.strptime(date_str, fmt)
            return dt.strftime("%d-%m-%y")
        except Exception:
            continue
    print(f"Error formatting date: {date_str}")
    return date_str 

In [275]:




cn=0
if table is None:
    raise Exception("No table found in the template slide.")

# Prepare a new presentation for output
final_prs = Presentation()
final_prs.slide_width = prs.slide_width
final_prs.slide_height = prs.slide_height

for row in ws.iter_rows(min_row=2, values_only=True):
    prs_temp = Presentation('Role.pptx')
    slide = prs_temp.slides[0]
    table = find_table(slide)
    # Fill table by mapping
    for idx, value in enumerate(row):
        header = excel_headers[idx]
        if header in ["DOJ", "DOR"]:
            # print("Ajit")
            value = format_date(value)
            # print(value)

        if header in excel_to_table_map:
            r, c = excel_to_table_map[header]
            table.cell(r, c).text = str(value) if value is not None else ""
            #print(f"Set cell ({r}, {c}) to '{value}' for header '{header}'")
    # Copy filled slide to final presentation
    # Usage:
    blank_slide_layout = final_prs.slide_layouts[6]
    new_slide = final_prs.slides.add_slide(blank_slide_layout)

    for shape in slide.shapes:
        if shape.shape_type == 19:  # 19 is MSO_SHAPE_TYPE.TABLE
            el = deepcopy(shape.element)
            new_slide.shapes._spTree.insert_element_before(el, 'p:extLst')
    # Optionally, handle other shapes (like images/text) as needed
    # cn += 1
    # if cn ==2:
    #     break

final_prs.save('Filled_Role.pptx')